In [1]:
df = spark.createDataFrame([[0, 33.3, -17.5],
                              [1, 40.4, -20.5],
                              [2, 28., -23.9],
                              [3, 29.5, -19.0],
                              [4, 32.8, -18.84]
                             ],
                              ["id","lat", "long"])

df.show()

+---+----+------+
 id| lat| long|
+---+----+------+
 0|33.3| -17.5|
 1|40.4| -20.5|
 2|28.0| -23.9|
 3|29.5| -19.0|
 4|32.8|-18.84|
+---+----+------+

In [2]:
from pyspark.ml.feature import VectorAssembler
vecAssembler = VectorAssembler(inputCols=["lat", "long"], outputCol="features")
new_df = vecAssembler.transform(df)
new_df.show()

+---+----+------+-------------+
 id| lat| long| features|
+---+----+------+-------------+
 0|33.3| -17.5| [33.3,-17.5]|
 1|40.4| -20.5| [40.4,-20.5]|
 2|28.0| -23.9| [28.0,-23.9]|
 3|29.5| -19.0| [29.5,-19.0]|
 4|32.8|-18.84|[32.8,-18.84]|
+---+----+------+-------------+

In [3]:
from pyspark.ml.clustering import KMeans

kmeans = KMeans(k=3, seed=1)  # 3 clusters here
model = kmeans.fit(new_df.select('features'))

In [4]:
transformed = model.transform(new_df)
transformed.show() 

+---+----+------+-------------+----------+
 id| lat| long| features|prediction|
+---+----+------+-------------+----------+
 0|33.3| -17.5| [33.3,-17.5]| 0|
 1|40.4| -20.5| [40.4,-20.5]| 1|
 2|28.0| -23.9| [28.0,-23.9]| 2|
 3|29.5| -19.0| [29.5,-19.0]| 0|
 4|32.8|-18.84|[32.8,-18.84]| 0|
+---+----+------+-------------+----------+

In [5]:
# Evaluate clustering by computing Within Set Sum of Squared Errors.
wssse = model.computeCost(new_df)
print("Within Set Sum of Squared Errors = " + str(wssse))

# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Within Set Sum of Squared Errors = 9.883733333332202
Cluster Centers: 
[ 31.86666667 -18.44666667]
[ 40.4 -20.5]
[ 28. -23.9]

In [6]:
#type 转换 for HW1 
data_list = [{'name': 'Alice', 'age': '1'}, 
     {'name': 'Mike', 'age': '2'},
     {'name': 'json', 'age': '3'},
    ]
 
df_new = spark.createDataFrame(data_list)

from pyspark.sql.types import DoubleType

changedTypedf = df_new.withColumn("age", df_new["age"].cast(DoubleType()))

changedTypedf.show()

/databricks/spark/python/pyspark/sql/session.py:355: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
 warnings.warn("inferring schema from dict is deprecated,"
+---+-----+
age| name|
+---+-----+
1.0|Alice|
2.0| Mike|
3.0| json|
+---+-----+